# Data Preparation

We need to represent and access info for all passengers: gender(weight), which group they belong to, if they have to transfert. We created a class Passenger to represent this data. We also created a class Group to reprenent each group (numer of women, men, transit time).



In [254]:
class Passenger:
    def __init__(self, group_index, passenger_id, description, transit_time=0):
        """
            group_index : 
            passenger_id : 
            description : men, women, wheelchair, child
            business_economic : class (Business, Economic)
            transit_time : optional parameter (inf if there is not transit time)
        
        """
        self.group_index = group_index
        self.passenger_id = passenger_id
        self.description = description
        # self.business_economic = business_economic
        self.transit_time = transit_time
        self.weight()
    
    def weight(self):
        if self.description == 'women':
            self.weight = 70
        elif self.description == 'men':
            self.weight = 85
        elif self.description == 'child':
            self.weight = 35
        else: 
            self.weight = 70 # need to check this value

In [255]:
import pandas as pd
import numpy as np
import os

In [256]:
passenger_id = 0

In [257]:
class Group:

    def __init__(self, id_group, num_women, num_men, transit_time=0):
        self.id_group = id_group
        self.num_women = num_women
        self.num_men = num_men
        self.transit_time = transit_time

        self.members = []

        global passenger_id

        for i in range(self.num_women):
            self.members.append(Passenger(
                self.id_group,
                passenger_id,
                'women',
                self.transit_time
            ))
            passenger_id+=1
        
        for i in range(self.num_men):
            self.members.append(Passenger(
                self.id_group,
                passenger_id,
                'men',
                self.transit_time
            ))
            passenger_id+=1
        
        return
    
    
        

In [258]:
def read_and_preprocess(file, date):
    df = pd.read_excel(file, date)
    
    df['TransitTime (min)'] = df['TransitTime'].apply(lambda x: float(str(x).split(':')[0])*60+float(str(x).split(':')[1]))

    df[['Femmes', 'Hommes', 'Enfants', 'WCHR', 'WCHB']] = df[['Femmes', 'Hommes', 'Enfants', 'WCHR', 'WCHB']].fillna(0)

    return df

def get_list_passengers(df):
    list_groups = {}

    for idx, row in df.iterrows():

        list_groups[int(idx)] = Group(
            id_group=int(idx),
            num_women=int(row['Femmes']),
            num_men=int(row['Hommes']),
            transit_time=row['TransitTime (min)']
        )
    
    return list_groups

In [259]:
df = read_and_preprocess('DataSeating - V3.xlsx', '21Oct')

In [260]:
df.head()

,Numéro du groupe,Femmes,Hommes,Enfants,WCHR,WCHB,Classe,TransitTime,TransitTime (min)
0,1,0.0,1.0,0.0,0.0,0.0,Y,01:15:00,75.0
1,2,1.0,0.0,0.0,0.0,0.0,Y,00:00:00,0.0
2,3,0.0,1.0,0.0,0.0,0.0,Y,00:00:00,0.0
3,4,1.0,0.0,0.0,0.0,0.0,Y,00:00:00,0.0
4,5,0.0,2.0,0.0,0.0,0.0,Y,05:25:00,325.0


In [261]:
groups_list = get_list_passengers(df)

In [262]:
passengers_list = list()

for g in groups_list.keys():
    passengers_list += groups_list[g].members

In [263]:
len(passengers_list)

151

In [264]:
passengers_list

# Creating the Airplane

In [265]:
airplane = dict()

airplane['xmax'] = 28
airplane['ymax'] = 7
airplane['corridor_y'] = 4
airplane['max_transition_row'] = 13

airplane['seats'] = dict()

# considering there is only business class
real_seats = []
corridor_seats = []
transition_seats = []

for x in range(1, airplane['xmax']+1):
    for y  in range(1, airplane['ymax']+1):
        if y != airplane['corridor_y']:
            real_seats.append([x, y])
        if y == airplane['corridor_y']:
            corridor_seats.append([x, y])
        if x <= airplane['max_transition_row'] and y != airplane['corridor_y']:
            transition_seats.append([x, y])

airplane['seats']['real'] = real_seats
airplane['seats']['corridor'] = corridor_seats
airplane['seats']['transition_possible_seats'] = transition_seats

del real_seats, corridor_seats, transition_seats

print(airplane)



{'xmax': 28, 'ymax': 7, 'corridor_y': 4, 'max_transition_row': 13, 'seats': {'real': [[1, 1], [1, 2], [1, 3], [1, 5], [1, 6], [1, 7], [2, 1], [2, 2], [2, 3], [2, 5], [2, 6], [2, 7], [3, 1], [3, 2], [3, 3], [3, 5], [3, 6], [3, 7], [4, 1], [4, 2], [4, 3], [4, 5], [4, 6], [4, 7], [5, 1], [5, 2], [5, 3], [5, 5], [5, 6], [5, 7], [6, 1], [6, 2], [6, 3], [6, 5], [6, 6], [6, 7], [7, 1], [7, 2], [7, 3], [7, 5], [7, 6], [7, 7], [8, 1], [8, 2], [8, 3], [8, 5], [8, 6], [8, 7], [9, 1], [9, 2], [9, 3], [9, 5], [9, 6], [9, 7], [10, 1], [10, 2], [10, 3], [10, 5], [10, 6], [10, 7], [11, 1], [11, 2], [11, 3], [11, 5], [11, 6], [11, 7], [12, 1], [12, 2], [12, 3], [12, 5], [12, 6], [12, 7], [13, 1], [13, 2], [13, 3], [13, 5], [13, 6], [13, 7], [14, 1], [14, 2], [14, 3], [14, 5], [14, 6], [14, 7], [15, 1], [15, 2], [15, 3], [15, 5], [15, 6], [15, 7], [16, 1], [16, 2], [16, 3], [16, 5], [16, 6], [16, 7], [17, 1], [17, 2], [17, 3], [17, 5], [17, 6], [17, 7], [18, 1], [18, 2], [18, 3], [18, 5], [18, 6], [18, 

# Defining the model

In [266]:
from gurobipy import *

In [267]:
m = Model('First Try')

## Decision Variables
The main decision variable here is the allocation A : binary variable that says for each passenger where he is located : 

A[x, y, p.passenger_id]

For instance, if Passenger 1 is at the place : width=2, height=3, then allocation[2,3,1]=1. Else, it equals 0.

There are also the satisfaction variables: depends on the number of neighbors of the same group you have [to be completed]

In [268]:
A = dict()

for x in range(airplane['xmax']+1):
    for y in range(airplane['ymax']+1):
        for p in passengers_list:
            A[x, y, p.passenger_id] = m.addVar(vtype=GRB.BINARY, name="A_{},{},{}".format(x,y,p.passenger_id))

## Constraints

### Barycenter

Separate the X and Y axis. Then separate each axis into 2 constraints: 1 lower constraint, 1 upper constraint, to fit into the defined barycenter area : 13<X<17 and 3<Y<5. To calculate the barycenter, we multipliy for each person its weight by its distance to the center of the airplane, multiplied by the allocation. The sum is then divided by the total weight (of every passengers). 

$\sum_{passenger = 1}^{p}[weight * A[x, y, p.passenger_id]*x  /total_{weight} \leq  1$ for i in range(width) for j in range(height)

$\sum_{passenger = 1}^{p}[weight * A[x, y, p.passenger_id]*x  /total_{weight} \geq -1$ for i in range(width) for j in range(height)

$\sum_{passenger = 1}^{p}[weight * A[x, y, p.passenger_id]*y  /total_{weight} \leq 2$ for i in range(width) for j in range(height)

$\sum_{passenger = 1}^{p}[weight * A[x, y, p.passenger_id]*y  /total_{weight} \geq -2$ for i in range(width) for j in range(height)

With x being the distance from the passenger to the center on the X axis and y being the distance from the passenger to the center on the Y axis


In [269]:
# Barycenter: x \in [13,17], y \in [3,5]
total_weight_passengers = sum(p.weight for p in passengers_list)

barycenter = dict()

barycenter['xinf'] = m.addConstr(quicksum(p.weight*x*A[x, y, p.passenger_id] for x,y in airplane['seats']['real'] for p in passengers_list) >= 13*total_weight_passengers, name="barycenter_x_inf")
barycenter['xsup'] = m.addConstr(quicksum(p.weight*x*A[x, y, p.passenger_id] for x,y in airplane['seats']['real'] for p in passengers_list) <= 17*total_weight_passengers, name="barycenter_x_sup")
barycenter['yinf'] = m.addConstr(quicksum(p.weight*y*A[x, y, p.passenger_id] for x,y in airplane['seats']['real'] for p in passengers_list) >= 3*total_weight_passengers, name="barycenter_y_inf")
barycenter['ysup'] = m.addConstr(quicksum(p.weight*y*A[x, y, p.passenger_id] for x,y in airplane['seats']['real'] for p in passengers_list) <= 5*total_weight_passengers, name="barycenter_y_sup")


### No passenger in the corridor

In [270]:
corridor = m.addConstr(quicksum(A[x,y,p.passenger_id] for x, y in airplane['seats']['corridor'] for p in passengers_list) == 0)

### At most 1 person per seat constraint: in the plane, there can't be more than 1 person in each seat.

$\sum_{passenger = 1}^{p}A[x, y, p.passenger_id]  \leq 1 $
for i in range(width) for j in range(height)

p being the number of passengers

In [271]:
passenger_seat_constr = dict()

for x, y in airplane['seats']['real']:
    passenger_seat_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] for p in passengers_list) <= 1, name='1 passenger at most at seat {},{}'.format(x,y))


### One seat per passenger

Everyone has 1 and only 1 seat constraint: each passenger has to be seated in only one place.

$$\sum_{i,j = 1}^{w,h}A[x, y, p.passenger_id] =1 $$

for p in passengers

With w the width (X), h the height (Y)

In [272]:
seat_passenger_constr = dict()

for x,y in airplane['seats']['real']:
    seat_passenger_constr[(x, y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] for p in passengers_list) == 1, name='1 seat per passenger {}'.format(p.passenger_id))

### Passengers in transition

Constraint of transfert passenger : if a passenger has to transfert, he needs to be in approximately the first third of the plane (X axis). We chose row 13, to be a bit more flexible. No constraint on the Y axis. If there is no transfert, the passenger can be anywhere.


 For p in passengers,  $$  p.transfert=1 \rightarrow A[x, y, p.passenger\_id][x] \leq 13 \;\;\;\;  \;\;\;$$

This will be implemented with an if condition before the constraint creation.

In [273]:
# transition_constr = dict()

# for p in passengers_list:
#     if p.transit_time > 0:
#         transition_constr[(p.passenger_id, x, y)] = m.addConstr()
# #         transition_constr[(p.passenger_id, x, y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] for x, y in airplane['seats']['transition_possible_seats'] for p in passengers_list) == 1, name='passenger {} in transition'.format(p.passenger_id))

### Satisfaction variables, "Neighboor from the same group" variables and auxiliary variables

For each position (x,y) in the plane, we have defined a group of 7 binary variables:

* __is_pair_y_the_same_group__ = 1 if the right-side neighboor of the person in (x,y) is from the same group as him. 0 otherwise or if any of the places is empty

* __is_pair_x_the_same_group__ = much like the previous variable, but for the neighboor in the back

* __aux1__ and __aux2__ = two auxiliary variables that will be used in some non-trivial constraints we will define.

* __c1__ = 1 if theres no one in either position (x,y) and (x,y+1) (will also be used for some non-trivial constraints)

* __c2__ = much like c1, but for (x+1,y)

And finally, the satisfaction variable S for the position (x,y). If there's no one in the position, this variable should assume the value 0. The objective function will try to maximize the sum of all S's while respecting some defining constraints. Although the definition of our problem presumed diferent criteria for evaluating the satisfaction level for people in small and big groups, we first tried to use a simplified version of the big group criterium: people across the corridor are considered neighboors and back/front neighboors count as half.



In [274]:
S = dict()
is_pair_y_the_same_group = dict() 
is_pair_x_the_same_group = dict()
aux1 = dict()
aux2 = dict()
c1 = dict()
c2 = dict()

for x, y in airplane['seats']['real']:
    S[(x,y)] = m.addVar(vtype=GRB.CONTINUOUS, name="satisfaction_{}_{}".format(x, y))
    aux1 [(x,y)] = m.addVar(vtype=GRB.BINARY, name="aux1_{}_{}".format(x, y))
    aux2 [(x,y)] = m.addVar(vtype=GRB.BINARY, name="aux2_{}_{}".format(x, y))
    c1 [(x,y)] = m.addVar(vtype=GRB.BINARY, name="c1_{}_{}".format(x, y))
    c2 [(x,y)] = m.addVar(vtype=GRB.BINARY, name="c2_{}_{}".format(x, y))
    
    # no right-side neighboor
    if y != 7:
        is_pair_y_the_same_group [(x,y)] = m.addVar(vtype=GRB.BINARY, name="y_pair_{}_{}".format(x, y))
    
    #no back neighboor
    if x != 28:
        is_pair_x_the_same_group [(x,y)] = m.addVar(vtype=GRB.BINARY, name="x_pair_{}_{}".format(x, y))

### constraints for setting up the "neighboors from the same group" variables

This was considered to be the real challenge in our approach's implementation: How to set the neighboor's variables without knowing who is where?

The logic behind our implementation is: 

We use the expression $$\sum_{p}^{passengers}A[x, y, p.passenger\_id] * p.group\_index $$ to find the number of the group in a position (x,y) and  use it to do the following checks:

$$ \textrm{if} \sum_{p}^{passengers}A[x, y, p.passenger\_id] * p.group\_index = \sum_{p}^{passengers}A[x, y+1, p.passenger\_id] * p.group\_index \quad \textrm{then} 
\quad isPairYtheSameGroup [(x,y)] = 1$$

$$ \textrm{if} \sum_{p}^{passengers}A[x, y, p.passenger\_id] * p.group\_index = \sum_{p}^{passengers}A[x+1, y, p.passenger\_id] * p.group\_index \quad \textrm{then} 
\quad isPairXtheSameGroup [(x,y)] = 1$$

For the case when $y = 3$, however, we should skip the corridor in $y = 4$ and sightly adapt the check to the following:

$$ \textrm{if} \sum_{p}^{passengers}A[x, y, p.passenger\_id] * p.group\_index = \sum_{p}^{passengers}A[x, y+2, p.passenger\_id] * p.group\_index \quad \textrm{then} 
\quad isPairYtheSameGroup [(x,y)] = 1$$


Implementing a constraint of type $x = b  \Rightarrow y = 1$ is not a simple task in linear programming. In order to do this, we used the solution proposed by Larry Snider's response in [a stack exchange post](https://or.stackexchange.com/questions/33/in-an-integer-program-how-i-can-force-a-binary-variable-to-equal-1-if-some-cond) which makes use of two auxiliary variables $z_1$ and $z_2$, that correspond to aux_1 and aux_2 in our implementation.


In [275]:
s_passenger_constr = dict()
is_pair_y1_the_same_group_constr = dict()
is_pair_x1_the_same_group_constr = dict()
is_pair_y2_the_same_group_constr = dict()
is_pair_x2_the_same_group_constr = dict()
is_pair_y3_the_same_group_constr = dict()
is_pair_x3_the_same_group_constr = dict()
is_pair_y4_the_same_group_constr = dict()
is_pair_x4_the_same_group_constr = dict()
is_pair_y5_the_same_group_constr = dict()
is_pair_x5_the_same_group_constr = dict()
is_pair_y6_the_same_group_constr = dict()
is_pair_x6_the_same_group_constr = dict()

cy1_constr = dict()
cy2_constr = dict()
cx1_constr = dict()
cx2_constr = dict()


for x, y in airplane['seats']['real']:
    if y != 7 and y != 3:
        
        # if (x,y) and (x,y+1) are from the same group and are occupied, then is_pair_y_the_same_group(x,y) = 1 and is 0 otherwise
        is_pair_y1_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y+1, p.passenger_id] * p.group_index for p in passengers_list) + 0.5 <= 100 * aux1[(x,y)], name='pair y definition1 {}_{}'.format(x, y))
        is_pair_y2_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y+1, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - 0.5 <= 100 *(1 - aux1[(x,y)]), name='pair y definition2 {}_{}'.format(x, y))
        is_pair_y3_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y+1, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + 0.5 <= 100 * aux2[(x,y)], name='pair y definition4 {}_{}'.format(x, y))
        is_pair_y4_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y+1, p.passenger_id] * p.group_index for p in passengers_list) - 0.5 <= 100 *(1 - aux2[(x,y)]), name='pair y definition5 {}_{}'.format(x, y))
        is_pair_y5_the_same_group_constr[(x,y)] = m.addConstr(aux1[(x,y)] + aux2[(x,y)] - 1 <= is_pair_y_the_same_group[(x,y)] + 100*(1 - c1[(x,y)]), name='pair y definition6 {}_{}'.format(x, y))
        is_pair_y6_the_same_group_constr[(x,y)] = m.addConstr(c1[(x,y)] >= is_pair_y_the_same_group[(x,y)], name='pair y definition7 {}_{}'.format(x, y))
        cy1_constr[(x,y)] = m.addConstr(100*quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + 100*quicksum(A[x, y+1, p.passenger_id] * p.group_index for p in passengers_list)  >= c1[(x,y)], name='cy definition1 {}_{}'.format(x, y))
        cy2_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + quicksum(A[x, y+1, p.passenger_id] * p.group_index for p in passengers_list)  <= 100*c1[(x,y)], name='cy definition2 {}_{}'.format(x, y))

    if x != 28:

        # if (x,y) and (x+1,y) are from the same group and are occupied, then is_pair_x_the_same_group(x,y) = 1 and is 0 otherwise
        is_pair_x1_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x+1, y, p.passenger_id] * p.group_index for p in passengers_list) + 0.5 <= 100 * aux1[(x,y)], name='pair x definition1 {}_{}'.format(x, y))
        is_pair_x2_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x+1, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - 0.5 <= 100 *(1 - aux1[(x,y)]), name='pair x definition2 {}_{}'.format(x, y))
        is_pair_x3_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x+1, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + 0.5 <= 100 * aux2[(x,y)], name='pair x definition4 {}_{}'.format(x, y))
        is_pair_x4_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x+1, y, p.passenger_id] * p.group_index for p in passengers_list) - 0.5 <= 100 *(1 - aux2[(x,y)]), name='pair x definition5 {}_{}'.format(x, y))
        is_pair_y5_the_same_group_constr[(x,y)] = m.addConstr(aux1[(x,y)] + aux2[(x,y)] - 1 <= is_pair_x_the_same_group[(x,y)] + 100*(1 - c2[(x,y)]), name='pair x definition6 {}_{}'.format(x, y))
        is_pair_x6_the_same_group_constr[(x,y)] = m.addConstr(c2[(x,y)] >= is_pair_x_the_same_group[(x,y)], name='pair x definition7 {}_{}'.format(x, y))
        cx1_constr[(x,y)] = m.addConstr(100*quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + 100*quicksum(A[x+1, y, p.passenger_id] * p.group_index for p in passengers_list)  >= c2[(x,y)], name='cx definition1 {}_{}'.format(x, y))
        cx2_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + quicksum(A[x+1, y, p.passenger_id] * p.group_index for p in passengers_list)  <= 100*c2[(x,y)], name='cx definition2 {}_{}'.format(x, y))


    if y == 3:
        
        # if (x,y) and (x,y+1) are separated by a corridor
        is_pair_y1_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y+2, p.passenger_id] * p.group_index for p in passengers_list) + 0.5 <= 100 * aux1[(x,y)], name='pair y definition1 {}_{}'.format(x, y))
        is_pair_y2_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y+2, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - 0.5 <= 100 *(1 - aux1[(x,y)]), name='pair y definition2 {}_{}'.format(x, y))
        is_pair_y3_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y+2, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + 0.5 <= 100 * aux2[(x,y)], name='pair y definition4 {}_{}'.format(x, y))
        is_pair_y4_the_same_group_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) - quicksum(A[x, y+2, p.passenger_id] * p.group_index for p in passengers_list) - 0.5 <= 100 *(1 - aux2[(x,y)]), name='pair y definition5 {}_{}'.format(x, y))
        is_pair_y5_the_same_group_constr[(x,y)] = m.addConstr(aux1[(x,y)] + aux2[(x,y)] - 1 <= is_pair_y_the_same_group[(x,y)] + 100*(1 - c1[(x,y)]), name='pair y definition6 {}_{}'.format(x, y))
        is_pair_y6_the_same_group_constr[(x,y)] = m.addConstr(c1[(x,y)] >= is_pair_y_the_same_group[(x,y)], name='pair y definition7 {}_{}'.format(x, y))
        cy1_constr[(x,y)] = m.addConstr(100*quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + 100*quicksum(A[x, y+2, p.passenger_id] * p.group_index for p in passengers_list)  >= c1[(x,y)], name='cy definition1 {}_{}'.format(x, y))
        cy2_constr[(x,y)] = m.addConstr(quicksum(A[x, y, p.passenger_id] * p.group_index for p in passengers_list) + quicksum(A[x, y+2, p.passenger_id] * p.group_index for p in passengers_list)  <= 100*c1[(x,y)], name='cy definition2 {}_{}'.format(x, y))



### constraints for the satisfaction levels

As previously mentioned, the objective function tries to maximize the sum of each position's satisfaction. In order to define the satisfaction for each position, we defined constraints that tie the satisfaction variables to the neighboors variables: we check the left, right, front and back neighboors and sum their binary values (if they are from the same group). But this sum is not as straightfoward, as it varies if the position is in a corner, in a border of the plane, or by the corridor.

For example, someone in the (1,1) position may have a right and back neighboor, but its front and left neighboors are not defined, and thats why we only sum the corresponding neighboors variables for the existing neighboors instead of all 4 of them. As we have to check this cases for every combination of extreme-case scenario, we made use of a number of if-else statements which made dificult the visualization of the general satisfation relation.

In [276]:
for x, y in airplane['seats']['real']:
    if y == 1:
        if x == 1:
            s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + 0.5*(is_pair_x_the_same_group[(x,y)]))
        else:
            if x == 28:
                s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + 0.5*(is_pair_x_the_same_group[(x-1,y)]))
            else:
                s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + 0.5*(is_pair_x_the_same_group[(x,y)] + is_pair_x_the_same_group[(x-1,y)]))
    else:
        if x == 1:
            if y == 7:
                s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y-1)] + 0.5*(is_pair_x_the_same_group[(x,y)] ))
            else:
                if y == 5:
                    s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + is_pair_y_the_same_group[(x,y-2)] + 0.5*(is_pair_x_the_same_group[(x,y)] ))
                else:
                    s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + is_pair_y_the_same_group[(x,y-1)] + 0.5*(is_pair_x_the_same_group[(x,y)] ))
        else:
            if y == 7:
                if x == 28:
                    s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y-1)] + 0.5*( is_pair_x_the_same_group[(x-1,y)]))
                else:
                    s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y-1)] + 0.5*(is_pair_x_the_same_group[(x,y)] + is_pair_x_the_same_group[(x-1,y)]))
            else:
                if x == 28:
                    if y == 5:
                        s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + is_pair_y_the_same_group[(x,y-2)] + 0.5*( is_pair_x_the_same_group[(x-1,y)]))
                    else:
                        s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + is_pair_y_the_same_group[(x,y-1)] + 0.5*( is_pair_x_the_same_group[(x-1,y)]))
                else:
                    if y == 5:
                        s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + is_pair_y_the_same_group[(x,y-2)] + 0.5*(is_pair_x_the_same_group[(x,y)] + is_pair_x_the_same_group[(x-1,y)]))
                    else:
                        s_passenger_constr[(x,y)] = m.addConstr(S[(x,y)] == is_pair_y_the_same_group[(x,y)] + is_pair_y_the_same_group[(x,y-1)] + 0.5*(is_pair_x_the_same_group[(x,y)] + is_pair_x_the_same_group[(x-1,y)]))
        
    

## Objective function

In [277]:
m.setObjective(quicksum(S[(x,y)] for x, y in airplane['seats']['real']), GRB.MAXIMIZE)

## Optimization

As can be seen in the following results, the objective function is being greatly overestimated and peoples allocation are not following the grouping patterns expected. We believe the error is coming from either the constraints defining the binary neighboors variables or the ones defining the satisfaction levels, and thus verificating those blocks are a high-priority task for the team. We are commited to solve this problem as soon as possible in order to keep the project's planned schedule on track. 

In [278]:
m.update()

m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2925 rows, 36174 columns and 704432 nonzeros
Model fingerprint: 0x277d1fa9
Variable types: 168 continuous, 36006 integer (36006 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 2e+05]
Found heuristic solution: objective -0.0000000
Presolve removed 507 rows and 13702 columns
Presolve time: 1.33s
Presolved: 2418 rows, 22472 columns, 276678 nonzeros
Variable types: 0 continuous, 22472 integer (22036 binary)

Root relaxation: objective 4.420000e+02, 1207 iterations, 0.05 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  442.00000    0  103   -0.00000  442.00000      -     -    1s
H    0  

In [279]:
placement = {}

for p in passengers_list:
    for x,y in airplane['seats']['real']:
        if A[x,y,p.passenger_id].x == 1:
            placement[p.passenger_id] = (x,y)  
            
for g in groups_list.keys():
    print("Group {} (transitTime : {} min):".format(g,groups_list[g].transit_time))
    for p in groups_list[g].members:
        if p.passenger_id in placement.keys():
            coord = placement[p.passenger_id]
            x,y = coord
            print("passenger {}, seat {}".format(p.passenger_id,coord))
    print("\n")    

Group 0 (transitTime : 75.0 min):
passenger 0, seat (28, 7)


Group 1 (transitTime : 0.0 min):
passenger 1, seat (28, 1)


Group 2 (transitTime : 0.0 min):
passenger 2, seat (28, 5)


Group 3 (transitTime : 0.0 min):
passenger 3, seat (24, 3)


Group 4 (transitTime : 325.0 min):
passenger 4, seat (10, 3)
passenger 5, seat (15, 1)


Group 5 (transitTime : 0.0 min):
passenger 6, seat (19, 1)


Group 6 (transitTime : 190.0 min):
passenger 7, seat (26, 6)


Group 7 (transitTime : 80.0 min):
passenger 8, seat (18, 2)
passenger 9, seat (16, 7)


Group 8 (transitTime : 140.0 min):
passenger 10, seat (3, 1)


Group 9 (transitTime : 0.0 min):
passenger 11, seat (24, 1)


Group 10 (transitTime : 235.0 min):
passenger 12, seat (25, 2)
passenger 13, seat (3, 2)


Group 11 (transitTime : 70.0 min):


Group 12 (transitTime : 0.0 min):
passenger 15, seat (25, 5)


Group 13 (transitTime : 0.0 min):
passenger 16, seat (10, 1)
passenger 17, seat (27, 5)


Group 14 (transitTime : 360.0 min):
passenger 18